<a href="https://colab.research.google.com/github/najdawi11/American_Health_and_Wellness-Dashboard/blob/main/Starter_Code_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import requests
import json
import numpy as np
from sodapy import Socrata

from pathlib import Path
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

import pydotplus
from IPython.display import Image


In [3]:
pip install sodapy

In [6]:
client = Socrata("data.cdc.gov", 'RR1JGCh5rZDwjeVYEvT9cDvj6')

results_2022 = client.get("epbn-9bv3", limit=500000)

results_2022_df = pd.DataFrame.from_records(results_2022)
results_2022_df = results_2022_df.drop(['data_value_footnote_symbol','data_value_footnote'], axis=1).dropna()
results_2022_df.head()

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,geolocation,locationid,categoryid,measureid,datavaluetypeid,short_question_text
0,2020,AL,Alabama,Abbeville,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,44.3,42.9,45.7,2688,"{'type': 'Point', 'coordinates': [-85.25278085...",0100124,HLTHOUT,OBESITY,CrdPrv,Obesity
1,2020,AL,Alabama,Andalusia,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,45.9,45.2,46.6,9015,"{'type': 'Point', 'coordinates': [-86.47806878...",0101708,HLTHOUT,OBESITY,CrdPrv,Obesity
2,2020,AL,Alabama,Baileyton,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.4,9.9,11.0,610,"{'type': 'Point', 'coordinates': [-86.60822915...",0103676,HLTHOUT,CASTHMA,CrdPrv,Current Asthma
3,2019,AL,Alabama,Beatrice,BRFSS,Prevention,Cholesterol screening among adults aged >=18 y...,%,Age-adjusted prevalence,87.3,85.9,88.6,301,"{'type': 'Point', 'coordinates': [-87.20904641...",0104900,PREVENT,CHOLSCREEN,AgeAdjPrv,Cholesterol Screening
4,2020,AL,Alabama,Black,BRFSS,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,4.2,3.6,4.6,207,"{'type': 'Point', 'coordinates': [-85.74279127...",0107120,HLTHOUT,STROKE,CrdPrv,Stroke


In [25]:

health_conditions = [
    'Cancer excluding Skin',
    'Coronary Heart Disease',
    'COPD',
    'Obesity',
    'Diabetes',
    'Health Insurance',
    'Annual Check-ups'
]


filtered_df_22 = results_2022_df[results_2022_df['short_question_text'].isin(health_conditions)]


print(filtered_df_22)


        year stateabbr statedesc locationname datasource         category  \
0       2020        AL   Alabama    Abbeville      BRFSS  Health Outcomes   
1       2020        AL   Alabama    Andalusia      BRFSS  Health Outcomes   
6       2020        AL   Alabama   Bon Secour      BRFSS  Health Outcomes   
13      2020        AL   Alabama      Chelsea      BRFSS  Health Outcomes   
22      2020        AL   Alabama    Evergreen      BRFSS  Health Outcomes   
...      ...       ...       ...          ...        ...              ...   
499981  2020        IA      Iowa     Meservey      BRFSS  Health Outcomes   
499983  2020        IA      Iowa       Monroe      BRFSS  Health Outcomes   
499994  2020        IA      Iowa      Merrill      BRFSS  Health Outcomes   
499995  2020        IA      Iowa        Ogden      BRFSS  Health Outcomes   
499999  2020        IA      Iowa    Muscatine      BRFSS  Health Outcomes   

                                                  measure data_value_unit  

In [26]:
filtered_df_22.head()

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,geolocation,locationid,categoryid,measureid,datavaluetypeid,short_question_text
0,2020,AL,Alabama,Abbeville,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,44.3,42.9,45.7,2688,"{'type': 'Point', 'coordinates': [-85.25278085...",0100124,HLTHOUT,OBESITY,CrdPrv,Obesity
1,2020,AL,Alabama,Andalusia,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,45.9,45.2,46.6,9015,"{'type': 'Point', 'coordinates': [-86.47806878...",0101708,HLTHOUT,OBESITY,CrdPrv,Obesity
6,2020,AL,Alabama,Bon Secour,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,33.0,31.9,34.1,999,"{'type': 'Point', 'coordinates': [-87.72715876...",0108272,HLTHOUT,OBESITY,CrdPrv,Obesity
13,2020,AL,Alabama,Chelsea,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,32.0,31.1,32.9,10519,"{'type': 'Point', 'coordinates': [-86.591535, ...",0114104,HLTHOUT,OBESITY,CrdPrv,Obesity
22,2020,AL,Alabama,Evergreen,BRFSS,Health Outcomes,Chronic obstructive pulmonary disease among ad...,%,Crude prevalence,11.0,10.2,11.8,3981,"{'type': 'Point', 'coordinates': [-86.9538785,...",0124808,HLTHOUT,COPD,CrdPrv,COPD


In [38]:

states_to_keep = ['California','Illinois', 'Indiana','Georgia','Alabama', 'Arkansas', 'Colorado','Arizona',
                  'Idaho', 'Hawaii', 'Connecticut', 'Delaware', 'Missouri', 'Minnesota', 'Maryland', 'Michigan', 'Mississippi', 'Kentucky', 'Louisiana', 'Distict of Columbia', 'Massachusetts', 'Maine', 'Alaska' ]




filtered_df22 = filtered_df_22[filtered_df_22['statedesc'].isin(states_to_keep)]


print(filtered_df_22)




        year stateabbr statedesc locationname datasource         category  \
0       2020        AL   Alabama    Abbeville      BRFSS  Health Outcomes   
1       2020        AL   Alabama    Andalusia      BRFSS  Health Outcomes   
6       2020        AL   Alabama   Bon Secour      BRFSS  Health Outcomes   
13      2020        AL   Alabama      Chelsea      BRFSS  Health Outcomes   
22      2020        AL   Alabama    Evergreen      BRFSS  Health Outcomes   
...      ...       ...       ...          ...        ...              ...   
499981  2020        IA      Iowa     Meservey      BRFSS  Health Outcomes   
499983  2020        IA      Iowa       Monroe      BRFSS  Health Outcomes   
499994  2020        IA      Iowa      Merrill      BRFSS  Health Outcomes   
499995  2020        IA      Iowa        Ogden      BRFSS  Health Outcomes   
499999  2020        IA      Iowa    Muscatine      BRFSS  Health Outcomes   

                                                  measure data_value_unit  

In [35]:
filtered_df22.head()

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,geolocation,locationid,categoryid,measureid,datavaluetypeid,short_question_text
0,2020,AL,Alabama,Abbeville,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,44.3,42.9,45.7,2688,"{'type': 'Point', 'coordinates': [-85.25278085...",0100124,HLTHOUT,OBESITY,CrdPrv,Obesity
1,2020,AL,Alabama,Andalusia,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,45.9,45.2,46.6,9015,"{'type': 'Point', 'coordinates': [-86.47806878...",0101708,HLTHOUT,OBESITY,CrdPrv,Obesity
6,2020,AL,Alabama,Bon Secour,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,33.0,31.9,34.1,999,"{'type': 'Point', 'coordinates': [-87.72715876...",0108272,HLTHOUT,OBESITY,CrdPrv,Obesity
13,2020,AL,Alabama,Chelsea,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,32.0,31.1,32.9,10519,"{'type': 'Point', 'coordinates': [-86.591535, ...",0114104,HLTHOUT,OBESITY,CrdPrv,Obesity
22,2020,AL,Alabama,Evergreen,BRFSS,Health Outcomes,Chronic obstructive pulmonary disease among ad...,%,Crude prevalence,11.0,10.2,11.8,3981,"{'type': 'Point', 'coordinates': [-86.9538785,...",0124808,HLTHOUT,COPD,CrdPrv,COPD


In [39]:
filtered_df22.value_counts('statedesc')

statedesc
California       14485
Illinois         13445
Indiana           6675
Georgia           6190
Alabama           5720
Arkansas          5255
Colorado          4175
Arizona           4045
Alaska            2620
Idaho             2120
Hawaii            1480
Connecticut       1420
Delaware           765
Missouri            46
Minnesota           38
Mississippi         26
Maryland            21
Michigan            21
Louisiana           20
Kentucky            17
Maine                5
Massachusetts        5
dtype: int64

In [37]:
filtered_df23.value_counts('statedesc')

statedesc
California              14485
Illinois                13445
Georgia                  6190
Alabama                  5720
Arkansas                 5255
Colorado                 4175
Arizona                  4045
Alaska                   2620
Indiana                  2591
Idaho                    2120
Hawaii                   1480
Connecticut              1420
Delaware                  765
Missouri                   23
Minnesota                  14
District of Columbia       10
Maryland                   10
Michigan                    9
Kentucky                    8
Louisiana                   5
Mississippi                 4
Maine                       2
Massachusetts               2
dtype: int64

In [29]:
filtered_df22.value_counts('short_question_text')

short_question_text
Obesity                   16465
Diabetes                  16433
COPD                      16376
Coronary Heart Disease    16365
Health Insurance          16348
dtype: int64

In [40]:
filtered_df23.value_counts('short_question_text')

short_question_text
Obesity                   12932
Diabetes                  12871
Coronary Heart Disease    12869
COPD                      12865
Health Insurance          12861
dtype: int64

In [ ]:
results_2022_df.value_counts('short_question_text')

short_question_text
Stroke                                      16635
Depression                                  16469
Obesity                                     16465
Arthritis                                   16461
Current Asthma                              16457
Diabetes                                    16433
Binge Drinking                              16395
High Blood Pressure                         16389
Mammography                                 16384
All Teeth Lost                              16380
COPD                                        16376
Cholesterol Screening                       16373
Coronary Heart Disease                      16365
Cancer (except skin)                        16360
Annual Checkup                              16360
Cervical Cancer Screening                   16357
Colorectal Cancer Screening                 16353
Dental Visit                                16349
Health Insurance                            16348
Current Smoking               

In [9]:
results_2022_df['year'] = results_2022_df['year'].astype(int)
results_2022_df['totalpopulation'] = results_2022_df['totalpopulation'].astype(int)
results_2022_df['data_value'] = results_2022_df['data_value'].astype(float)
results_2022_df['low_confidence_limit'] = results_2022_df['low_confidence_limit'].astype(float)
results_2022_df['high_confidence_limit'] = results_2022_df['high_confidence_limit'].astype(float)
results_2022_df.dtypes

year                       int64
stateabbr                 object
statedesc                 object
locationname              object
datasource                object
category                  object
measure                   object
data_value_unit           object
data_value_type           object
data_value               float64
low_confidence_limit     float64
high_confidence_limit    float64
totalpopulation            int64
geolocation               object
locationid                object
categoryid                object
measureid                 object
datavaluetypeid           object
short_question_text       object
dtype: object

In [10]:
search_value = ['Arthritis']
cervical_2022_df = results_2022_df[results_2022_df['short_question_text'].isin(search_value)]
cervical_2022_df.head()
cervical_2022_df.value_counts('statedesc')

search_value_2 = ['Alabama']
AL_cervical_2022_df = cervical_2022_df[cervical_2022_df['statedesc'].isin(search_value_2)]

search_value_3 = ['Age-adjusted prevalence']
age_AL_cervical_2022_df = AL_cervical_2022_df[AL_cervical_2022_df['data_value_type'].isin(search_value_3)]
age_AL_cervical_2022_df.head()

# only_Berry = age_AL_cervical_2022_df.loc[age_AL_cervical_2022_df['locationname']=='Berry',:]
# # print(only_Berry)

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,geolocation,locationid,categoryid,measureid,datavaluetypeid,short_question_text
10,2020,AL,Alabama,Carlton,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,33.4,32.0,34.8,65,"{'type': 'Point', 'coordinates': [-87.8385235,...",0112112,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis
17,2020,AL,Alabama,Dauphin Island,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,27.6,26.0,29.1,1238,"{'type': 'Point', 'coordinates': [-88.19395005...",0119744,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis
32,2020,AL,Alabama,Guntersville,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,30.8,30.0,31.6,8231,"{'type': 'Point', 'coordinates': [-86.30843425...",0132416,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis
73,2020,AL,Alabama,Prichard,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,33.3,32.9,33.7,22664,"{'type': 'Point', 'coordinates': [-88.13010143...",0162496,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis
86,2020,AL,Alabama,Snead,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,31.6,30.4,32.8,837,"{'type': 'Point', 'coordinates': [-86.39007806...",0171280,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis


In [11]:
results_2023 = client.get("eav7-hnsx", limit=500000)

results_2023_df = pd.DataFrame.from_records(results_2023)
results_2023_df = results_2023_df.drop(['data_value_footnote_symbol','data_value_footnote',':@computed_region_bxsw_vy29',':@computed_region_he4y_prf8'], axis=1).dropna()
results_2023_df.head()

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,geolocation,locationid,categoryid,measureid,datavaluetypeid,short_question_text
0,2020,AK,Alaska,Kiana,BRFSS,Health Outcomes,All teeth lost among adults aged >=65 years,%,Crude prevalence,38.5,29.0,48.0,347,"{'type': 'Point', 'coordinates': [-160.4343638...",0239300,HLTHOUT,TEETHLOST,CrdPrv,All Teeth Lost
1,2021,AK,Alaska,Koliganek,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Crude prevalence,22.0,18.5,25.7,209,"{'type': 'Point', 'coordinates': [-157.2259091...",0241500,HLTHOUT,ARTHRITIS,CrdPrv,Arthritis
2,2021,AK,Alaska,Kongiganak,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Crude prevalence,23.5,20.2,27.0,439,"{'type': 'Point', 'coordinates': [-162.8830767...",0241610,HLTHOUT,ARTHRITIS,CrdPrv,Arthritis
3,2021,AK,Alaska,Lakes,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,36.7,32.5,41.2,8364,"{'type': 'Point', 'coordinates': [-149.3066764...",0242832,HLTHOUT,OBESITY,CrdPrv,Obesity
4,2021,AK,Alaska,Mountain Village,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,47.3,39.2,56.1,813,"{'type': 'Point', 'coordinates': [-163.7209368...",0251180,HLTHOUT,OBESITY,CrdPrv,Obesity


In [15]:

health_conditions = [
    'Cancer excluding Skin',
    'Coronary Heart Disease',
    'COPD',
    'Obesity',
    'Diabetes',
    'Health Insurance',
    'Annual Check-ups'
]


filtered_df23 = results_2023_df[results_2023_df['short_question_text'].isin(health_conditions)]


print(filtered_df23)

        year stateabbr statedesc      locationname datasource  \
3       2021        AK    Alaska             Lakes      BRFSS   
4       2021        AK    Alaska  Mountain Village      BRFSS   
6       2021        AK    Alaska            Palmer      BRFSS   
7       2021        AK    Alaska        Petersburg      BRFSS   
8       2021        AK    Alaska   Russian Mission      BRFSS   
...      ...       ...       ...               ...        ...   
499939  2021        IN   Indiana           Liberty      BRFSS   
499948  2021        IN   Indiana             Mecca      BRFSS   
499973  2021        IN   Indiana        Plainfield      BRFSS   
499984  2021        IN   Indiana             Monon      BRFSS   
499993  2021        IN   Indiana            Oxford      BRFSS   

               category                                            measure  \
3       Health Outcomes               Obesity among adults aged >=18 years   
4       Health Outcomes               Obesity among adults aged

In [17]:
filtered_df23.head()

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,geolocation,locationid,categoryid,measureid,datavaluetypeid,short_question_text
3,2021,AK,Alaska,Lakes,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,36.7,32.5,41.2,8364,"{'type': 'Point', 'coordinates': [-149.3066764...",0242832,HLTHOUT,OBESITY,CrdPrv,Obesity
4,2021,AK,Alaska,Mountain Village,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,47.3,39.2,56.1,813,"{'type': 'Point', 'coordinates': [-163.7209368...",0251180,HLTHOUT,OBESITY,CrdPrv,Obesity
6,2021,AK,Alaska,Palmer,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,34.7,30.7,39.0,5937,"{'type': 'Point', 'coordinates': [-149.1146766...",0258660,HLTHOUT,OBESITY,CrdPrv,Obesity
7,2021,AK,Alaska,Petersburg,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,31.0,24.4,38.4,2948,"{'type': 'Point', 'coordinates': [-132.8721371...",0260310,HLTHOUT,OBESITY,CrdPrv,Obesity
8,2021,AK,Alaska,Russian Mission,BRFSS,Health Outcomes,Obesity among adults aged >=18 years,%,Crude prevalence,46.6,38.5,55.4,312,"{'type': 'Point', 'coordinates': [-161.3566777...",0265700,HLTHOUT,OBESITY,CrdPrv,Obesity


In [24]:
filtered_df23.

TypeError: ignored

In [18]:
results_2023_df['year'] = results_2023_df['year'].astype(int)
results_2023_df['totalpopulation'] = results_2023_df['totalpopulation'].astype(int)
results_2023_df['data_value'] = results_2023_df['data_value'].astype(float)
results_2023_df['low_confidence_limit'] = results_2023_df['low_confidence_limit'].astype(float)
results_2023_df['high_confidence_limit'] = results_2023_df['high_confidence_limit'].astype(float)
results_2023_df.dtypes

year                       int64
stateabbr                 object
statedesc                 object
locationname              object
datasource                object
category                  object
measure                   object
data_value_unit           object
data_value_type           object
data_value               float64
low_confidence_limit     float64
high_confidence_limit    float64
totalpopulation            int64
geolocation               object
locationid                object
categoryid                object
measureid                 object
datavaluetypeid           object
short_question_text       object
dtype: object

In [19]:
search_value = ['Arthritis']
cervical_2023_df = results_2023_df[results_2023_df['short_question_text'].isin(search_value)]
cervical_2023_df.head()
cervical_2023_df.value_counts('statedesc')

search_value_2 = ['Alabama']
AL_cervical_2023_df = cervical_2023_df[cervical_2023_df['statedesc'].isin(search_value_2)]

search_value_3 = ['Age-adjusted prevalence']
age_cervical_AL_2023_df = AL_cervical_2023_df[AL_cervical_2023_df['data_value_type'].isin(search_value_3)]
age_cervical_AL_2023_df.head(5)

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,geolocation,locationid,categoryid,measureid,datavaluetypeid,short_question_text
737,2021,AL,Alabama,Beaverton,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,30.7,26.2,35.4,201,"{'type': 'Point', 'coordinates': [-88.02177782...",0104948,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis
961,2021,AL,Alabama,Fayette,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,29.9,25.9,34.3,4619,"{'type': 'Point', 'coordinates': [-87.83073716...",0125840,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis
1034,2021,AL,Alabama,Douglas,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,30.0,25.9,34.5,744,"{'type': 'Point', 'coordinates': [-86.32243761...",0121328,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis
1169,2021,AL,Alabama,Belle Fontaine,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,27.3,24.3,30.3,608,"{'type': 'Point', 'coordinates': [-88.11321149...",0105212,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis
1192,2021,AL,Alabama,Grayson Valley,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,28.0,25.3,30.8,5724,"{'type': 'Point', 'coordinates': [-86.64136264...",0131348,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis


In [20]:
merged_age_Cervical_AL = pd.merge(age_AL_cervical_2022_df, age_cervical_AL_2023_df, on='locationname', how='inner')

merged_age_Cervical_AL.head(5)

,year_x,stateabbr_x,statedesc_x,locationname,datasource_x,category_x,measure_x,data_value_unit_x,data_value_type_x,data_value_x,...,data_value_y,low_confidence_limit_y,high_confidence_limit_y,totalpopulation_y,geolocation_y,locationid_y,categoryid_y,measureid_y,datavaluetypeid_y,short_question_text_y
0,2020,AL,Alabama,Carlton,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,33.4,...,31.7,27.4,36.2,65,"{'type': 'Point', 'coordinates': [-87.8385235,...",0112112,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis
1,2020,AL,Alabama,Dauphin Island,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,27.6,...,27.3,24.4,30.5,1238,"{'type': 'Point', 'coordinates': [-88.19395005...",0119744,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis
2,2020,AL,Alabama,Guntersville,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,30.8,...,28.3,24.4,32.4,8231,"{'type': 'Point', 'coordinates': [-86.30843425...",0132416,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis
3,2020,AL,Alabama,Prichard,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,33.3,...,31.8,28.9,34.9,22664,"{'type': 'Point', 'coordinates': [-88.13010143...",0162496,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis
4,2020,AL,Alabama,Snead,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Age-adjusted prevalence,31.6,...,31.3,27.1,35.6,837,"{'type': 'Point', 'coordinates': [-86.39007806...",0171280,HLTHOUT,ARTHRITIS,AgeAdjPrv,Arthritis


In [21]:
new_merged_df = merged_age_Cervical_AL.drop(['year_x', 'stateabbr_x', 'statedesc_x', 'datasource_x',
       'category_x', 'measure_x', 'data_value_unit_x', 'data_value_type_x',
       'low_confidence_limit_x', 'high_confidence_limit_x',
       'totalpopulation_x', 'geolocation_x', 'locationid_x', 'categoryid_x',
       'measureid_x', 'datavaluetypeid_x', 'short_question_text_x', 'year_y',
       'stateabbr_y', 'statedesc_y', 'datasource_y', 'category_y', 'measure_y',
       'data_value_unit_y', 'data_value_type_y',
       'low_confidence_limit_y', 'high_confidence_limit_y',
       'totalpopulation_y', 'geolocation_y', 'locationid_y', 'categoryid_y',
       'measureid_y', 'datavaluetypeid_y', 'short_question_text_y'], axis=1)

new_merged_df.head()

,locationname,data_value_x,data_value_y
0,Carlton,33.4,31.7
1,Dauphin Island,27.6,27.3
2,Guntersville,30.8,28.3
3,Prichard,33.3,31.8
4,Snead,31.6,31.3


In [12]:
new_merged_df['y'] = pd.Series(dtype=float)

new_merged_df

,locationname,data_value_x,data_value_y,y
0,Carlton,33.4,31.7,NaN
1,Dauphin Island,27.6,27.3,NaN
2,Guntersville,30.8,28.3,NaN
3,Prichard,33.3,31.8,NaN
4,Snead,31.6,31.3,NaN
...,...,...,...,...
564,Rainbow City,26.3,27.1,NaN
565,Nanafalia,31.2,31.1,NaN
566,Yellow Bluff,33.5,31.9,NaN
567,Woodville,30.3,30.0,NaN


In [13]:
for index, row in new_merged_df.iterrows():
    if (row['data_value_x'] < row['data_value_y']):
        new_merged_df.at[index, 'y'] = 1
    else:
        new_merged_df.at[index, 'y'] = 0

new_merged_df

,locationname,data_value_x,data_value_y,y
0,Carlton,33.4,31.7,0.0
1,Dauphin Island,27.6,27.3,0.0
2,Guntersville,30.8,28.3,0.0
3,Prichard,33.3,31.8,0.0
4,Snead,31.6,31.3,0.0
...,...,...,...,...
564,Rainbow City,26.3,27.1,1.0
565,Nanafalia,31.2,31.1,0.0
566,Yellow Bluff,33.5,31.9,0.0
567,Woodville,30.3,30.0,0.0


In [14]:
X = cervical_2023_df.copy()
X.drop(["low_confidence_limit","high_confidence_limit", "stateabbr", "statedesc", "locationname", "category", "datasource", "measure", "data_value_unit", "geolocation", "data_value_type", "locationid", "categoryid", "measureid", "datavaluetypeid", "short_question_text"], axis=1, inplace=True)
X.head()

,year,data_value,totalpopulation
1,2021,22.0,209
2,2021,23.5,439
11,2021,25.5,556
18,2021,18.2,5395
20,2021,23.6,821


In [15]:
y = cervical_2023_df["data_value"].values


from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [19]:
y

array([124, 139, 159, ..., 200, 137, 157])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [21]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
model1 = tree.DecisionTreeClassifier()

In [24]:
model1 = model1.fit(X_train_scaled, y_train)

In [25]:
predictions = model1.predict(X_test_scaled)

In [26]:
acc_score = accuracy_score(y_test, predictions)

print(f"Accuracy Score : {acc_score}")
# print("Classification Report")
# print(classification_report(y_test, predictions))

Accuracy Score : 0.9928770517187984


In [27]:
from sklearn.ensemble import RandomForestClassifier
model2 = RandomForestClassifier(n_estimators=500, random_state=78)

In [28]:
model2 = model2.fit(X_train_scaled, y_train.ravel())

In [29]:
y_pred = model2.predict(X_test_scaled)

In [30]:
# cm = confusion_matrix(y_test, y_pred)
# cm_df = pd.DataFrame(
#     cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
# )

# Calculating the accuracy score
acc_score = accuracy_score(y_test, y_pred)

print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.7559615980179623


In [31]:
from sklearn.metrics import confusion_matrix, classification_report
confusion_matrix(y_pred,y_test)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [32]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          17       0.00      0.00      0.00         0
          18       0.00      0.00      0.00         0
          21       0.00      0.00      0.00         0
          22       0.00      0.00      0.00         1
          26       0.00      0.00      0.00         0
          27       0.00      0.00      0.00         1
          29       0.00      0.00      0.00         0
          30       0.00      0.00      0.00         0
          33       0.00      0.00      0.00         0
          37       0.00      0.00      0.00         0
          38       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [33]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)

In [34]:
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [35]:
y_pred = knn.predict(X_test_scaled)

In [36]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         0
           7       0.50      1.00      0.67         1
           9       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         1
          21       0.00      0.00      0.00         1
          26       0.00      0.00      0.00         1
          27       0.00      0.00      0.00         0
          28       0.00      0.00      0.00         0
          29       0.00      0.00      0.00         1
          30       0.00      0.00      0.00         1
          33       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [37]:
search_value = ['OBESITY']
obesity_2022_df = results_2022_df[results_2022_df['measureid'].isin(search_value)]
obesity_2022_df.head()
obesity_2022_df.count()

year                     16465
stateabbr                16465
statedesc                16465
locationname             16465
datasource               16465
category                 16465
measure                  16465
data_value_unit          16465
data_value_type          16465
data_value               16465
low_confidence_limit     16465
high_confidence_limit    16465
totalpopulation          16465
geolocation              16465
locationid               16465
categoryid               16465
measureid                16465
datavaluetypeid          16465
short_question_text      16465
dtype: int64

In [38]:
search_value = ['OBESITY']
obesity_2023_df = results_2023_df[results_2023_df['measureid'].isin(search_value)]
obesity_2023_df.head()
obesity_2023_df.count()

year                     12932
stateabbr                12932
statedesc                12932
locationname             12932
datasource               12932
category                 12932
measure                  12932
data_value_unit          12932
data_value_type          12932
data_value               12932
low_confidence_limit     12932
high_confidence_limit    12932
totalpopulation          12932
geolocation              12932
locationid               12932
categoryid               12932
measureid                12932
datavaluetypeid          12932
short_question_text      12932
dtype: int64

In [39]:
search_value = ['OBESITY']
obesity_2020_df = results_2020_df[results_2020_df['measureid'].isin(search_value)]
obesity_2020_df.head()
obesity_2020_df.count()

NameError: ignored

In [ ]:
search_value = ['OBESITY']
obesity_2021_df = results_2021_df[results_2021_df['measureid'].isin(search_value)]
obesity_2021_df.head()
obesity_2021_df.count()

Combining 2020, 2021, 2022, and 2023 Obesity Dataframes in one dataframe using concat function.

In [ ]:
obesity_df = pd.concat([obesity_2020_df, obesity_2021_df, obesity_2022_df, obesity_2023_df], axis=0)
obesity_df


Filtered Age-Adjusted Prevalence

In [ ]:
search_value = ['Age-adjusted prevalence']
age_adjusted_obesity_df = obesity_df[obesity_df['data_value_type'].isin(search_value)]
age_adjusted_obesity_df.head()

Using KNN Model

In [ ]:
X = age_adjusted_obesity_df.copy()
X.drop(['data_value','stateabbr','statedesc','locationname','datasource','category','measure','data_value_unit',
        'data_value_type','geolocation','locationid','categoryid','measureid','datavaluetypeid','short_question_text'], axis=1, inplace=True)
X

In [ ]:
y = age_adjusted_obesity_df["data_value"].ravel()

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)

In [ ]:
model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = model.predict(X_test_scaled)

In [ ]:
y_pred

In [ ]:
confusion_matrix(y_pred,y_test)

In [ ]:
print(classification_report(y_pred,y_test))

Using Random Forest Method

In [ ]:
X = age_adjusted_obesity_df.copy()
X.drop(['data_value','stateabbr','statedesc','locationname','datasource','category','measure','data_value_unit',
        'data_value_type','geolocation','locationid','categoryid','measureid','datavaluetypeid','short_question_text'], axis=1, inplace=True)
X

In [ ]:
y = age_adjusted_obesity_df["data_value"].values.reshape(-1,1)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=5000, random_state=78)

In [ ]:
model = model.fit(X_train_scaled, y_train.ravel())

In [ ]:
y_pred = model.predict(X_test_scaled)

In [ ]:
# Calculating the accuracy score
acc_score = accuracy_score(y_test, y_pred)
acc_score

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
confusion_matrix(y_pred,y_test)

In [ ]:
print(classification_report(y_pred,y_test))

Using Neural Network

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=78)

X_scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
import sklearn as skl
import tensorflow as tf

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=4))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [ ]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=20,
    hyperband_iterations=2
)

In [ ]:
tuner.search(X_train_scaled, y_train, epochs=20,validation_data=(X_test_scaled, y_test))